In [89]:
import requests
# If you are using a Jupyter notebook, uncomment the following line.
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib import patches
from io import BytesIO
import json


import http.client, urllib.request, urllib.parse, urllib.error, base64

In [46]:
# Define General Global Variables
GROUP_NAME = "hackathon_atl"

# Define Face-API Specific Global Variables
FACE_API_KEY1 = "2d697fcd49e34ceaacce26f321034221"
FACE_API_URL = 'https://eastus2.api.cognitive.microsoft.com/face/v1.0'
HEADERS = {
    'Ocp-Apim-Subscription-Key': FACE_API_KEY1,
    'Content-Type': 'application/octet-stream'
}
PARAMS = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'true',
    'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,' +
    'emotion,hair,makeup,occlusion,accessories,blur,exposure,noise'
}

# 1. Create the Person Group

In [23]:
def createPersonGroup():
    HEADERS = {
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': FACE_API_KEY1,
    }
    PARAMS = urllib.parse.urlencode({
        'personGroupId': GROUP_NAME
    })
    BODY = '{"name": "%s"}' % GROUP_NAME

    try:
        conn = http.client.HTTPSConnection('eastus2.api.cognitive.microsoft.com')
        conn.request("PUT", "/face/v1.0/persongroups/{%s}?%s" % (GROUP_NAME, PARAMS), "%s" % (BODY), HEADERS)
        response = conn.getresponse()
        data = response.read()
        print(data)
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

# 2. Insert People into Person Group

In [55]:
def insertPersonIntoGroup(person_name, group_name):
    HEADERS = {
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': FACE_API_KEY1,
    }
    
    PARAMS = urllib.parse.urlencode({
        'personGroupId': group_name
    })
    
    BODY = '{"name":"%s"}' % (person_name)
    
    try:
        conn = http.client.HTTPSConnection('eastus2.api.cognitive.microsoft.com')
        conn.request("POST", "/face/v1.0/persongroups/{%s}/persons?%s" % (group_name, PARAMS), "%s" % (BODY), HEADERS)
        response = conn.getresponse()
        data = response.read()
        print(data)
        conn.close()
        return data
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    return None
    
insertPersonIntoGroup("Obama", GROUP_NAME)

b'{"personId":"8fb4eda3-de46-45b6-8818-8b1aa8329ecb"}'


b'{"personId":"8fb4eda3-de46-45b6-8818-8b1aa8329ecb"}'

# 3. Submit Images to Train Face On

In [56]:
Obama_ID = "f9a6fb5d-2c81-40c0-9cb0-e9980d62371c"

#import os
#for filename in os.listdir('.\\Obama'):
#    print(filename)

image_data = open(".\\Obama\\barack-obama.jpeg", "rb").read()
str_data = str(base64.b64decode(image_data))
str_data

'b\'$R\\x05\\x13\\x18\\x9f \\x88\\x82jz\\\'\\t\\xa9\\xe8\\x9cC\\x92\\xd45\\xc2\\x85\\xeb\\\'\\xa2\\xdfHu\\xd1\\x9d\\xdb\\xe2\\xbd\\xd3m6\\xdfN\\x85\\xe7\\xaez\\xd3]4<\\xd5t\\xd7ix\\xe7m5\\xef]\\xb4\\xdb_t\\xeb\\x9dv\\xd3^\\xf5\\xdbM\\xb5\\xf7N\\xb9\\xd6\\x07\\x9d\\xd7_7\\xd3_4\\xd3M8\\xf0Q\\x05\\xefM\\xb4\\xd2i\\x9f\\xffo\\x0b!"\\x08Q#4\\xd3M\\x1c\\xd9\\xd7\\xf5\\xdd\\x1d7\\xd3]4\\x04\\x91\\xc7\\x01\\xda\\x1bx#\\x00v\\x86\\xdev-\\xf5\\x01\\x06\\xaa\\xd8\\x14[\\xdf\\x8a\\xc2I\\xcb9\\r7D\\xb7\\xa5\\x1e\\xb8U_\\xbf\\xb1\\x96\\x83\\x0eu\\x01\\x06\\xaa\\xd8\\x14wn\\xb0\\x92r\\xce9\\r7DS\\xab^\\xb8U_\\xbf\\xb1\\x96\\x83\\x02ma\\xdd\\xf7mo\\xf2\\xbf\\x92\\xa8#\\xa4\\xa9\\r\\x7f\\xbc~\\xed&\\xe2W6\\xcf\\xbc&\\xba\\xfe\\x1c\\x1aA\\x15 tM\\x1c\\x84\\xc3l\\xb38\\x9c.\\xccr\\x833\\xc50\\xe7\\xc1\\xce\\xff\\n\\x1b\\\\\\xa1\\x8e\\x02#\\x0b\\x80jF\\x15\\xd4\\xa0\\x99\\x0c\\x064\\xde\\xc0\\xe7\\x8e\\xf2\\x85\\xf0>\\xbe\\x8a\\x0b\\x94\\xb97"\\x88j\''

In [70]:
def submitPersonImages(person_id, image_data):
    # Declare header data
    headers = {
        # Request headers
        'Content-Type': 'application/octet-stream',
        'Ocp-Apim-Subscription-Key': FACE_API_KEY1,
    }
    
    # Obtain full URL Path
    path = FACE_API_URL + "/persongroups/%s/persons/%s/persistedFaces" % (GROUP_NAME, person_id)
    
    try:
        response = requests.post(path, headers=headers, data=image_data)
        faces = response.json()
        print(faces)
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
# Loop through Obama picture directory.
img_path = './Obama/'
for filename in os.listdir(img_path):
    if filename.endswith('.jpg') or filename.endswith('.jpeg'):
        image_data = open(img_path + filename, "rb").read()
        submitPersonImages(Obama_ID, image_data)
        

{'persistedFaceId': '00ea2201-46c3-4a28-a74a-7363c50c5e1d'}
{'persistedFaceId': '7c81112f-6c1a-4989-93c5-b2a6c4fec009'}
{'persistedFaceId': '5143e16c-caf9-40a1-8b21-a0a9a78dbb9f'}
{'persistedFaceId': 'ef43e17a-85b0-49ac-a8da-ee32bb0ad2cd'}


# 4. Train Person Group

In [78]:
def function():
    return
function()

In [84]:
def trainPersonGroup():   
    headers = {
        # Request headers
        'Ocp-Apim-Subscription-Key': FACE_API_KEY1,
    }

    try:
        conn = http.client.HTTPSConnection('eastus2.api.cognitive.microsoft.com')
        conn.request("POST", "/face/v1.0/persongroups/%s/train" % (GROUP_NAME), "{body}", headers)
        response = conn.getresponse()
        data = response.read()
        print(data)
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    return
        
trainPersonGroup()

b''


# 5. Identify Obama

In [98]:
def detectFace(image_data):
    # Declare header data
    headers = {
        # Request headers
        'Content-Type': 'application/octet-stream',
        'Ocp-Apim-Subscription-Key': FACE_API_KEY1,
    }
   #detect[?returnFaceId][&returnFaceLandmarks][&returnFaceAttributes]
    params = ({
        # Request parameters
        'returnFaceId': 'true',
        'returnFaceLandmarks': 'false',
        'returnFaceAttributes': 'emotion',
    })
    
    path = FACE_API_URL + '/detect'
    
    try:
        response = requests.post(path, headers=headers, params=params, data=image_data)
        faces = response.json()
        #print(faces)
        return faces
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
    return None

In [99]:
def identifyPerson(face_id):
    # Declare header data
    headers = {
        # Request headers
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': FACE_API_KEY1,
    }
    
    body = {
        "personGroupId": GROUP_NAME,
        "faceIds": [face_id],
        "maxNumOfCandidatesReturned": 1,
        "confidenceThreshold": 0.5
    }
    
    # Obtain full URL Path
    path = FACE_API_URL + "/identify"
    
    try:
        response = requests.post(path, headers=headers, data=json.dumps(body))
        faces = response.json()
        #print(faces)
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

In [100]:
def getFaceIdentification(image):
    image_data = open(image, "rb").read()
    face_detection = detectFace(image_data)
    face_identity = identifyPerson(face_detection[0]['faceId'])
    return face_detection, face_identity
    
# Loop through Obama picture directory.
img_path = './Obama/'
for filename in os.listdir(img_path):
    if filename.endswith('.jpg') or filename.endswith('.jpeg'):
        print(getFaceIdentification(img_path + filename))

([{'faceId': 'e68724b2-aae9-4ae6-b71e-bb71df6af517', 'faceRectangle': {'top': 56, 'left': 29, 'width': 103, 'height': 103}, 'faceAttributes': {'emotion': {'anger': 0.0, 'contempt': 0.0, 'disgust': 0.0, 'fear': 0.0, 'happiness': 1.0, 'neutral': 0.0, 'sadness': 0.0, 'surprise': 0.0}}}], None)
([{'faceId': 'fc85ce0a-8204-4293-8470-323fef1bde55', 'faceRectangle': {'top': 247, 'left': 462, 'width': 508, 'height': 508}, 'faceAttributes': {'emotion': {'anger': 0.0, 'contempt': 0.001, 'disgust': 0.0, 'fear': 0.0, 'happiness': 0.333, 'neutral': 0.666, 'sadness': 0.0, 'surprise': 0.0}}}], None)
([{'faceId': '11d13d47-6b41-43c2-b0f8-61090023de6f', 'faceRectangle': {'top': 287, 'left': 513, 'width': 455, 'height': 455}, 'faceAttributes': {'emotion': {'anger': 0.0, 'contempt': 0.001, 'disgust': 0.0, 'fear': 0.0, 'happiness': 0.656, 'neutral': 0.343, 'sadness': 0.0, 'surprise': 0.0}}}], None)
([{'faceId': '77596fd2-75ad-45ad-813b-6f5260ea4d7a', 'faceRectangle': {'top': 89, 'left': 219, 'width': 194,

In [ ]:
#image_folder_path = "C:\\hackathon\\voicepassword\\IMG_0909\\IMG_0909_012.jpg"
#image_data = open(image_path, "rb").read()